<a href="https://colab.research.google.com/github/adityapatil4141/Deep-learning/blob/main/keras_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data importing

In [3]:
# Importing basic necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
# Importing dataset
df = pd.read_csv("/content/drive/MyDrive/work files /diabetes.csv")

In [5]:
# Dataset look
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
# Dataset coorelation
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [7]:
# Splitting dataset in x and y
x = df.drop(columns=['Outcome']).values
y = df['Outcome'].values


In [8]:
# Scaling x data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [9]:
# x shape
x.shape

(768, 8)

In [10]:
# splitting dataset in training and testing dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

# basic model without hyperparameter tuning

In [11]:
# Importing libraries
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [12]:
# Building model
model = Sequential()

model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
# Fitting model
model.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
20/20 [==============================] - 2s 30ms/step - loss: 0.7096 - accuracy: 0.5277 - val_loss: 0.6540 - val_accuracy: 0.6039
Epoch 2/10
20/20 [==============================] - 0s 8ms/step - loss: 0.6549 - accuracy: 0.6596 - val_loss: 0.6127 - val_accuracy: 0.7273
Epoch 3/10
20/20 [==============================] - 0s 9ms/step - loss: 0.6166 - accuracy: 0.7068 - val_loss: 0.5811 - val_accuracy: 0.7532
Epoch 4/10
20/20 [==============================] - 0s 8ms/step - loss: 0.5878 - accuracy: 0.7345 - val_loss: 0.5580 - val_accuracy: 0.7727
Epoch 5/10
20/20 [==============================] - 0s 8ms/step - loss: 0.5644 - accuracy: 0.7459 - val_loss: 0.5379 - val_accuracy: 0.7857
Epoch 6/10
20/20 [==============================] - 0s 8ms/step - loss: 0.5448 - accuracy: 0.7508 - val_loss: 0.5226 - val_accuracy: 0.7987
Epoch 7/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5290 - accuracy: 0.7557 - val_loss: 0.5099 - val_accuracy: 0.8052
Epoch 8/10
20/20 [=

In [ ]:
#accuracy=80%

In [14]:
# Install keras tuner
# Keras tuner is a library which help in hyperparameter tuning
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [15]:
# Importing kerastuner library
import kerastuner as kt

<ipython-input-15-faf94310e1d7>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


# Hyperparameter tuning for best optimizer

In [16]:
# This code snippet is used if the tuner.search is giving this output "./untitled_project/tuner0.json' "
# whenever we use
import os

if os.path.exists('./untitled_project/tuner0.json'):
  os.remove('./untitled_project/tuner0.json')  # Remove tuner state file


In [17]:
# Hyperparameter tuning for best optimizer

In [18]:
# building a function. with "hyperparameter = hp" as parameter
def build_model(hp):
  model2 = Sequential()

  model2.add(Dense(32,activation='relu',input_dim=8))
  model2.add(Dense(1,activation='sigmoid'))

  model2.compile(loss='binary_crossentropy',optimizer = hp.Choice('optimizer',values=['adam','rmsprop','adadelta','sgd']),metrics=['accuracy'])
  # Above hp.choice will random values and build model.
  # hp here is function parameter, 'optimizer' is any string and values=[] is list of optimizer which you want model to check
  return model2
  # Return model



In [19]:
# Importing RandomSearch from kerastuner, Workds like gridsearchcv
from kerastuner.tuners import RandomSearch

In [20]:
# RandomSearch takes, build_model(Function), objective(what we want to reduce/increase), max_trials= how many times we will interate over model
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [21]:
# search here is a future of tuner object.
# tuner.search helps to search best hyperparameter
tuner.search(x_train,y_train,validation_data=(x_test,y_test),epochs=5)

Trial 4 Complete [00h 00m 05s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 15s


In [22]:
# Get best hyperparameter
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [23]:
# Fetching best model
model2 = tuner.get_best_models(num_models=1)[0]

In [24]:
# model summary
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# Fitting Model
# initial_epoch = from where we want to start epochs again, we had 5 epoch in tuner.search so we select 5
model2.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(x_test,y_test))

Epoch 6/100
20/20 [==============================] - 2s 27ms/step - loss: 0.4964 - accuracy: 0.7736 - val_loss: 0.4851 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4866 - accuracy: 0.7834 - val_loss: 0.4773 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4792 - accuracy: 0.7736 - val_loss: 0.4718 - val_accuracy: 0.7792
Epoch 9/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4729 - accuracy: 0.7785 - val_loss: 0.4689 - val_accuracy: 0.7792
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4685 - accuracy: 0.7866 - val_loss: 0.4654 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4650 - accuracy: 0.7866 - val_loss: 0.4641 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 12ms/step - loss: 0.4619 - accuracy: 0.7883 - val_loss: 0.4626 - val_accuracy: 0.7987
Epoch 13

In [ ]:
#accuracy = 80%

# Hyperparameter tuning for best neurons count

In [26]:
# Building function

def build_model(hp):
  model = Sequential()

  units = hp.Int('units',min_value=8,max_value=128) #units=any string, also can use step_size = .....
  # to find best neuron count

  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

  return model

In [27]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir',project_name='patil')
# directory = creates a directory where a json file containing information and details about RandomSearch,
# project_name = folder name inside directory is where details of this particular code snippet will be included

In [28]:
# If used directory and project_name this code is not required
# import os

# if os.path.exists('./untitled_project/tuner0.json'):
#   os.remove('./untitled_project/tuner0.json')  # Remove tuner state file


In [29]:
# Searching best hyperparameters
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 20s


In [30]:
# Best hyperparameters
tuner.get_best_hyperparameters()[0].values

{'units': 85}

In [31]:
# Beat model
model = tuner.get_best_models(num_models=1)[0]

In [32]:
# MOdel fitting on more epochs
model.fit(x_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 3ms/step - loss: 0.4827 - accuracy: 0.7736
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4738 - accuracy: 0.7785
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4678 - accuracy: 0.7736
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4634 - accuracy: 0.7769
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4597 - accuracy: 0.7785
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4570 - accuracy: 0.7834
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4559 - accuracy: 0.7801
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4521 - accuracy: 0.7866
Epoch 15/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4509 - accuracy: 0.7834
Epoch 16/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4494 - accuracy: 0.7818
Epo

In [33]:
# 82 % accuracy

# How to select no. of layers

In [35]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(61,activation='relu',input_dim=8)) #To avoid complexity creating a layer with input. 61 = best no. of neurons we found above

  # Below code is to find best no. of layers
  for i in range(hp.Int('layers',min_value=1,max_value=10)): # Using for loop to find best no. of layers in range of 1-10
    model.add(Dense(61,activation='relu'))# 61 = best no. of neurons we found above

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

  return model

In [36]:
# tuner object
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='layers')

In [37]:
# Searching
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 15s


In [38]:
# Fetching best hyperparameters
tuner.get_best_hyperparameters()[0].values

{'layers': 5}

In [39]:
# fetching best model
model = tuner.get_best_models(num_models=1)[0]

In [40]:
# fitting model
model.fit(x_train,y_train,epochs=100,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 7s 66ms/step - loss: 0.4255 - accuracy: 0.7769 - val_loss: 0.4713 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 16ms/step - loss: 0.4108 - accuracy: 0.7997 - val_loss: 0.4515 - val_accuracy: 0.8052
Epoch 9/100
20/20 [==============================] - 0s 15ms/step - loss: 0.4021 - accuracy: 0.8176 - val_loss: 0.4541 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 15ms/step - loss: 0.3820 - accuracy: 0.8322 - val_loss: 0.4528 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 13ms/step - loss: 0.3807 - accuracy: 0.8241 - val_loss: 0.4508 - val_accuracy: 0.8117
Epoch 12/100
20/20 [==============================] - 0s 17ms/step - loss: 0.3690 - accuracy: 0.8257 - val_loss: 0.4652 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 13ms/step - loss: 0.3608 - accuracy: 0.8534 - val_loss: 0.4671 - val_accuracy: 0.7922
Ep

In [41]:
# Accuracy = 74%

# All hyperparameters tuning at once.

In [42]:
# Building function

def build_model(hp):
  model = Sequential()
  counter = 0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter == 0:
      model.add(
          Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8), # + str(i) to change name everytime in every iteration
              activation=hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid']),# To find best activation
              input_dim=8
              )
      )
      model.add(
          Dropout(
              hp.Choice('dropout'+str(i) ,values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]) #+ str(i) to change name everytime in every iteration
          )
      )

    else:
      model.add(
          Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8), #+ str(i) to change name everytime in every iteration
              activation=hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid']),
              )
          )
      model.add(
          Dropout(
              hp.Choice('dropout'+str(i) ,values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]) #+ str(i) to change name everytime in every iteration
          )
      )
    counter+=1


  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['adam','rmsprop','sgd','adadelta']),loss='binary_crossentropy',metrics='accuracy')
  # Finding best optimizer above

  return model


In [43]:
# Tuner object
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='done')

In [44]:
# searching
tuner.search(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 13s


In [45]:
#fetching best hyperparameter
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 72,
 'activation0': 'relu',
 'dropout0': 0.1,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1}

In [46]:
# Fetching best model
model = tuner.get_best_models(num_models=1)[0]

In [47]:
# Fitting model
model.fit(x_train,y_train,epochs=100,initial_epoch=5,validation_data=(x_test,y_test))

Epoch 6/100
20/20 [==============================] - 2s 19ms/step - loss: 0.5301 - accuracy: 0.7524 - val_loss: 0.4833 - val_accuracy: 0.7922
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5238 - accuracy: 0.7590 - val_loss: 0.4838 - val_accuracy: 0.7792
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5151 - accuracy: 0.7508 - val_loss: 0.4814 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5336 - accuracy: 0.7427 - val_loss: 0.4812 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5000 - accuracy: 0.7557 - val_loss: 0.4804 - val_accuracy: 0.7727
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5282 - accuracy: 0.7362 - val_loss: 0.4807 - val_accuracy: 0.7727
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4883 - accuracy: 0.7720 - val_loss: 0.4792 - val_accuracy: 0.7792
Epoch 13/

In [ ]:
#accuracy= 79%